In [24]:
# %pip install pymorphy2==0.9.1

Note: you may need to restart the kernel to use updated packages.


In [1]:
# %pip install autocorrect==2.6.1

     |████████████████████████████████| 622 kB 2.7 MB/s eta 0:00:01
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=b0201096fcbac6b5aa71651e3f4ffcc6b10ec8c6cfd938af378bfe8ffe6aa40a
  Stored in directory: /home/user/.cache/pip/wheels/ab/0f/23/3c010c3fd877b962146e7765f9e9b08026cac8b035094c5750
Successfully built autocorrect
Note: you may need to restart the kernel to use updated packages.


In [1]:
import re
from typing import List
from collections import defaultdict, Counter

# from autocorrect import Speller
# from pymorphy2 import MorphAnalyzer
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

## Читаем датасеты и выполняем обзор имеющихся данных

In [2]:
df_train = pd.read_csv('train_dataset_train.csv')
df_test = pd.read_csv('test_dataset_test.csv')

In [3]:
df_train.head(10)

,RecordNo,Название книги,Автор,Ссылка на литрес,Рейтинг,Количество оценок,Количество отзывов,Имя читателя,Оценка книги читателем (из 5 баллов),Отзыв,Лайки на отзыв,Дислайки на отзыв,Релевантность,Таксономия релевантные,Таксономия не релевантные,Длина отзыва,Ценности
0,6145,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,Айгуль Ляпина,5.0,Рекомендую книгу в прочтению/прослушиванию. Ес...,0,3,0,0,0,0,0
1,7006,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.6,24719,2103,Olga T,5.0,"Удивительно, что сейчас возникает ТАКАЯ литера...",0,1,0,0,0,0,1
2,1124,Дети мои,Гузель Яхина,https://www.litres.ru/guzel-yahina/deti-moi/,4.4,8032,702,Кирилл Чириков,5.0,"Душевно, жизненно, чувственно, проникновенно!!...",0,0,0,1,0,0,1
3,4716,ПОСТ,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/post/,3.9,430,48,Kimetov Oleg,5.0,Очень понравилось. Причем понравилось больше «...,2,7,0,1,0,0,0
4,2946,ПОСТ,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/post-...,4.4,832,113,Сергей Никитин,5.0,Круто. Автор и он же чтец – гений. Прослушал н...,1,0,0,0,0,0,0
5,592,Санькя,Захар Прилепин,https://www.litres.ru/zahar-prilepin/sankya-16...,4.1,274,37,Кирилл К,3.0,"Вообще Прилепин пишет хорошо. Хорошо читается,...",0,1,0,0,0,0,0
6,7635,Обитель,Захар Прилепин,https://www.litres.ru/zahar-prilepin/obitel/,4.5,1916,171,Сергей Киряков,5.0,"Это вторая книга атора, которую я прочитал. Ни...",0,0,0,1,0,0,1
7,7226,Батюшки мои,Валентин Курбатов,https://www.litres.ru/valentin-kurbatov/batush...,4.7,3,6,Елена Тимофеева,NaN,церковь в начале 90-х В центре повествования П...,1,1,1,1,0,0,1
8,6194,Зулейха открывает глаза,Гузель Яхина,https://www.litres.ru/guzel-yahina/zuleyha-otk...,4.7,3922,408,yuliya_bv,5.0,Роман произвёл очень сильное впечатление. Авто...,1,3,1,1,0,0,1
9,3276,Текст,Дмитрий Глуховский,https://www.litres.ru/dmitriy-gluhovskiy/tekst/,4.4,7276,622,shilina.anna,5.0,"На одном дыхании. Без соплей, без чернухи, как...",0,1,0,0,0,0,1


In [4]:
print(np.mean(df_train['Релевантность']))
print(np.mean(df_train['Таксономия релевантные']))
print(np.mean(df_train['Таксономия не релевантные']))
print(np.mean(df_train['Длина отзыва']))
print(np.mean(df_train['Ценности']))

0.21864021492043811
0.512295928910932
0.12688572019012193
0.23475924777846663
0.5420541434180616


## Подготовка данных

In [5]:
# Функции для обработки текста

RU_LETTERS = re.compile('[а-яё]+')
CLEAN_HTML = re.compile('<.*?>')
spell = Speller('ru')
morph = MorphAnalyzer()

# Удяляем html-теги, в нижний регистр, оставляем только русские буквы. Разбиваем на токены.
# Удаляем слова короче 3-х символов как неинформативные.
def format_text(raw_text: str) -> str:
    # Убираем html-теги
    txt = re.sub(CLEAN_HTML, '', raw_text)
    
    # Приводим к нижнему регистру
    txt = txt.lower()
    
    # Оставляем только слова из русских букв
    txt = " ".join(RU_LETTERS.findall(txt))
    
    # Удаляем неинформативные слова длиной менее 2-х букв.
    words_filtered = []
    for word in txt.split():
        if len(word) > 2:
            words_filtered.append(word)
    return ' '.join(words_filtered)


# Приводим все слова к нормальной форме. Удаляем неинформативные части речи: предлоги, союзы,частицы, междометия.
def format_norm(txt: str) -> str:
    words = []
    for word in txt.split():
        m = morph.parse(word)[0]
        if m.tag.POS not in ('PREP', 'CONJ', 'PRCL', 'INTJ'):
            words.append(m.normal_form)
    return ' '.join(words)
    

In [7]:
# Подготовка данных для заданного датасета.

def format_df_inplace(df: pd.DataFrame) -> None:
    # "Причёсываем" Отзыв.
    df['Отзыв format'] = df['Отзыв'].apply(format_text)
    print('Format: Done')
    # Исправляем опечатки.
    df['Отзыв spell'] = df['Отзыв format'].apply(spell)
    print('Spell: Done')
    # Приводим к нормальной форме
    df['Отзыв norm'] = df['Отзыв spell'].apply(format_norm)
    print('Norm: Done')

In [8]:
# Еще раз читаем исходные данные на случай, если в ходе экспериментов в них уже что-то было поломано

df_train = pd.read_csv('train_dataset_train.csv')
df_test = pd.read_csv('test_dataset_test.csv')

format_df_inplace(df_train)
format_df_inplace(df_test)

# Так как подготовка знимает много времени, сохраним результаты чтобы в следующий раз сразу загрузить уже подготовленные данные.
df_train.to_csv('train_dataset_prepared.csv')
df_test.to_csv('test_dataset_prepared.csv')

Format: Done
Spell: Done
Norm: Done
Format: Done
Spell: Done
Norm: Done


In [2]:
df_train = pd.read_csv('train_dataset_prepared_2.csv', index_col=0, keep_default_na=False)
df_test = pd.read_csv('test_dataset_prepared_2.csv', index_col=0, keep_default_na=False)

In [3]:
df_train['msglen'] = [len(txt.split()) for txt in df_train['Отзыв format']]
df_test['msglen'] = [len(txt.split()) for txt in df_test['Отзыв format']]


Используем классический вариант алгоритма Наивный Байесовский классификатор. 

Построим множество слов из текстов всех отзывов тренировочного датасета, обозначим его $A$. Пусть $|A| = n$.

Далее выберем и зафиксируем обращение, свойства которого мы хотим определить. Множество слов из текста этого обрашения обозначим $A_0$.

Для произвольного случайно выбранного отзыва введем следующие обозначения для событий.

$C_k$ - отзыв принадлежит категории $k$, $k = 1, 0$.

Для каждого слова $i \in A$ рассмотрим событие $W_i$, которое определяется следующим образом.
Если слово $i \in A_0$, то $W_i$ состоит в том, что слово $i$ содержится в тексте случайно выбранного отзыва.
Если слово $i \notin A_0$, то $W_i$ состоит в том, что слово $i$ НЕ содержится в тексте случайно выбранного отзыва.

Тогда по теореме Байеса:

$$
P(C_k|W_1\cap W_2\cap ... \cap W_n) = \frac{P(W_1\cap W_2\cap ... \cap W_n|C_k)P(C_k)}{P(W_1\cap W_2\cap ... \cap W_n)}.
$$

Делая наивное предположение о независимости событий, получаем:

$$
P(C_k|W_1\cap W_2\cap ... \cap W_n) = \frac{P(W_1|C_k)P(W_2|C_k) ... P(W_n|C_k) P(C_k)}{P(W_1)P(W_2) ... P(W_n)}.
$$

Оцениваем шанс попадания зафиксированного события в категорию $k$ как
$$
\frac{P(C_k|W_1\cap W_2\cap ... \cap W_n)}{\sum_{q = 0}^{1} P(C_q|W_1\cap W_2\cap ... \cap W_n) }.
$$

И выбираем значение признака (0 или 1), соответствующую максимуму этой величины.

In [91]:
# Универсальный (с точки зрения выбора текстовых полей для классификации) класс для выполнения классификации Наивным Байесом.

class NaiveBayes:
    def __init__(self, train_text_list: np.array, train_cat_list: np.array):
        if not (len(train_text_list) == len(train_cat_list)):
            raise Exception('Не совпадает длина входных списков')
        train_cat_list = train_cat_list[~pd.isna(train_text_list)]
        train_text_list = train_text_list[~pd.isna(train_text_list)]
            
        self.train_cnt = len(train_text_list)
        
        # Подсчитываем в скольких записях встречается каждое слово во всех категориях.
        # Сколько раз встречается каждое слово в каждой категории.
        # А также количество записей с каждой категорией

        self.word_cnt = defaultdict(int)
        self.word_cnt_by_cat = defaultdict(int)
        self.train_cnt_by_cat = [0, 0]
        for ind in range(len(train_text_list)):
            txt = train_text_list[ind]
            cat = train_cat_list[ind]
            for word in set(txt.split()):
                self.word_cnt[word] += 1
                self.word_cnt_by_cat[(cat, word)] += 1
            self.train_cnt_by_cat[cat] += 1
        print(len(self.word_cnt))
        self.word_cnt = dict((key, self.word_cnt[key]) for key in self.word_cnt if self.word_cnt[key] >= 15)
        
    def get_word_cnt_in_cat(self, cat: int, word: str) -> int:
        return self.word_cnt_by_cat.get((cat, word)) or 0

    def get_word_cnt_not_in_cat(self, cat: int, word: str) -> int:
        return (self.train_cnt_by_cat[cat] - self.word_cnt_by_cat.get((cat, word), 0)) or 0
    
    def get_cat_probs(self, txt: str) -> List[float]:
        if pd.isna(txt):
            return np.array([0, 1], dtype=np.longdouble)
        chance = np.array([0, 0], dtype=np.longdouble)
        word_set = Counter(txt.split())
        chance[0] = self.train_cnt_by_cat[0] / self.train_cnt
        chance[1] = self.train_cnt_by_cat[1] / self.train_cnt
        for word in self.word_cnt:
            p = word_set[word]
            if p > 0:
                chance[0] *= self.get_word_cnt_in_cat(0, word) / self.train_cnt_by_cat[0]
                chance[1] *= self.get_word_cnt_in_cat(1, word) / self.train_cnt_by_cat[1]
            else:
                chance[0] *= self.get_word_cnt_not_in_cat(0, word) / self.train_cnt_by_cat[0]
                chance[1] *= self.get_word_cnt_not_in_cat(1, word) / self.train_cnt_by_cat[1]
                

        sum_chance = sum(chance)
        return [ch / sum_chance for ch in chance]


In [63]:
# Выполняем предсказание по всем полям

def predict_all(df_train, df_test):
    for field_name in ("Релевантность", "Таксономия релевантные", "Таксономия не релевантные",
                        "Длина отзыва", "Ценности"):
        print(field_name)
        naive_bayes = NaiveBayes(df_train['Отзыв norm'], 
                                 df_train[field_name].values)
        df_test[field_name] = [np.argmax(naive_bayes.get_cat_probs(txt)) for txt in df_test['Отзыв norm'].values]

In [77]:
# Выполняем предсказание по всем полям

def predict_all(df_train, df_test):
    
    for field_name in ("Релевантность", "Таксономия релевантные", "Таксономия не релевантные"):
        print(field_name)
        naive_bayes = NaiveBayes(df_train['Отзыв norm'], 
                                 df_train[field_name].values)
        df_test[field_name] = [np.argmax(naive_bayes.get_cat_probs(txt)) for txt in df_test['Отзыв norm'].values]

    fileld_name = 'Длина отзыва'
    print(fileld_name)
    naive_bayes = NaiveBayes(df_train['Отзыв norm'].values, 
                             df_train[fileld_name].values)
    df_test[fileld_name] = [(np.argmax(naive_bayes.get_cat_probs(txt)) if msglen <= 81 else 0) 
                            for txt, msglen in zip(
                                df_test['Отзыв norm'].values, 
                                df_test.msglen.values)]

    fileld_name = 'Ценности'
    print(fileld_name)
    naive_bayes = NaiveBayes(df_train['Отзыв norm'].values, 
                             df_train[fileld_name].values)
    df_test[fileld_name] = [(np.argmax(naive_bayes.get_cat_probs(txt)) if msglen <= 137 else 1) 
                            for txt, msglen in zip(
                                df_test['Отзыв norm'].values, 
                                df_test.msglen.values)]


In [70]:
# Вычисление метрики по всем полям

def get_metrics(df_test, df_predict):
    res = dict()
    for field_name in ("Релевантность", "Таксономия релевантные", "Таксономия не релевантные",
                        "Длина отзыва", "Ценности"):
        res[field_name] = recall_score(df_test[field_name].values, df_predict[field_name].values, average='macro')
    res['Итог'] = 0.2 * np.sum(list(res.values()))
    return res

In [72]:
df_test_oper = pd.DataFrame({'RecordNo': df_train['RecordNo'].values, 
                             'msglen': df_train.msglen.values, 
                             'Отзыв norm': df_train['Отзыв norm']})

In [89]:
predict_all(df_train, df_test_oper)

Релевантность
14086
Таксономия релевантные
14086
Таксономия не релевантные
14086
Длина отзыва
14086
Ценности
14086


In [92]:
get_metrics(df_train, df_test_oper)

{'Релевантность': 0.7279977641665696,
 'Таксономия релевантные': 0.9306830768284095,
 'Таксономия не релевантные': 0.949373976061523,
 'Длина отзыва': 0.7610340683039636,
 'Ценности': 0.903472183035106,
 'Итог': 0.8545122136791143}

In [80]:
df_test_oper_fin = pd.DataFrame({'RecordNo': df_test['RecordNo'].values, 
                                 'msglen': df_test.msglen.values, 
                                 'Отзыв norm': df_test['Отзыв norm'].values})

In [81]:
predict_all(df_train, df_test_oper_fin)

Релевантность
14086
Таксономия релевантные
14086
Таксономия не релевантные
14086


/tmp/ipykernel_5361/1337150737.py:53: RuntimeWarning: invalid value encountered in longdouble_scalars
  return [ch / sum_chance for ch in chance]


Длина отзыва
14086
Ценности
14086


In [85]:
df_test_oper_fin[['RecordNo', "Релевантность", "Таксономия релевантные", "Таксономия не релевантные",
                        "Длина отзыва", "Ценности"]].to_csv("result4.csv", index = False)